In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from qtm.constants import RYDBERG, ELECTRONVOLT
from qtm.kpts import KList
from qtm.lattice import RealLattice
from qtm.crystal import BasisAtoms, Crystal
from qtm.pseudo import UPFv2Data
# from qtm.klist import gen_monkhorst_pack_grid
from qtm.gspace import GSpace
from qtm.mpi import QTMComm
from qtm.dft import DFTCommMod, scf

from qtm.io_utils.dft_printers import print_scf_status

from qtm import qtmconfig
from qtm.logger import qtmlogger
# qtmconfig.fft_backend = 'mkl_fft'

from mpi4py.MPI import COMM_WORLD
comm_world = QTMComm(COMM_WORLD)
dftcomm = DFTCommMod(comm_world)

# Lattice
reallat = RealLattice.from_alat(alat=10.2,  # Bohr
                                a1=[-0.5,  0.0,  0.5],
                                a2=[ 0.0,  0.5,  0.5],
                                a3=[-0.5,  0.5,  0.0])

# Atom Basis
si_oncv = UPFv2Data.from_file('Si_ONCV_PBE-1.2.upf')
si_atoms = BasisAtoms.from_alat('Si', si_oncv, 28.086, reallat,
                               [[0.875, 0.875, 0.875], [0.125, 0.125, 0.125]])

crystal = Crystal(reallat, [si_atoms, ])  # Represents the crystal


# Generating k-points from a Monkhorst Pack grid (reduced to the crystal's IBZ)
mpgrid_shape = (4, 4, 4)
x = np.linspace(0,1,mpgrid_shape[0], endpoint=False)
y = np.linspace(0,1,mpgrid_shape[1], endpoint=False)
z = np.linspace(0,1,mpgrid_shape[2], endpoint=False)
xx,yy,zz = np.meshgrid(x,y,z, indexing="ij")
kcryst = np.vstack([xx.flatten(),yy.flatten(),zz.flatten()])

kpts = KList(recilat=crystal.recilat, k_coords=kcryst, k_weights=np.ones(kcryst.shape[1])/kcryst.shape[1])

# mpgrid_shift = (True,True,True)
# kpts = gen_monkhorst_pack_grid(crystal, mpgrid_shape, mpgrid_shift)
# print(kpts.numkpts)
# -----Setting up G-Space of calculation-----
ecut_wfn = 5 * RYDBERG
# NOTE: In future version, hard grid (charge/pot) and smooth-grid (wavefun)
# can be set independently
ecut_rho = 4 * ecut_wfn
grho = GSpace(crystal.recilat, ecut_rho)
gwfn = grho

# -----Spin-polarized (collinear) calculation-----
is_spin, is_noncolin = False, False
# Starting with asymmetric spin distribution else convergence may yield only
# non-magnetized states
mag_start = [0.0]
numbnd = 30  # Ensure adequate # of bands if system is not an insulator

occ = 'fixed'
smear_typ = 'gauss'
e_temp = 0.0001 * RYDBERG

conv_thr = 1E-8 * RYDBERG
diago_thr_init = 1E-2 * RYDBERG

In [3]:
from qtm.constants import ELECTRONVOLT_HART
from qtm.kpts import KList


kpts = KList(recilat=crystal.recilat, k_coords=kpts.k_cryst, k_weights=np.ones(kpts.k_cryst.shape[1])/kpts.k_cryst.shape[1])

out = scf(dftcomm, crystal, kpts, grho, gwfn,
          numbnd, is_spin, is_noncolin,
          rho_start=mag_start, occ_typ=occ, smear_typ='gauss', e_temp=e_temp,
          conv_thr=conv_thr, diago_thr_init=diago_thr_init,
          iter_printer=print_scf_status,
          ret_vxc=True)

scf_converged, rho, l_wfn_kgrp, en, vxc = out


print("SCF Routine has exited")
print(qtmlogger)

print(vxc/ELECTRONVOLT_HART)

# for wfn in l_wfn_kgrp:
#     wfn.evc_gk/=(np.linalg.norm(wfn.evc_gk, axis=-1, keepdims=True))
    # wfn.evl*=2
    # wfn.normalize()

Iteration # 1, Run Time:   5.9 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1171e-02 Ry
Avg Diago Iterations: 7.7
Diago Threshold     : 1.00e-02 Ry

Total Energy:          -15.96754455 Ry

      one-el:            4.83802701 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89767082 eV
    LU Level:            6.86611554 eV
----------------------------------------

Iteration # 1, Run Time:   8.2 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1010e-02 Ry
Avg Diago Iterations: 2.6
Diago Threshold     : 7.63e-04 Ry

Total Energy:          -15.96792936 Ry

      one-el:            4.83764220 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89675665 eV
    LU Level:            6.86552231 eV
----------------------------------------

Iteration # 2, Run Time:  10

/home1/agrimsharma/codes/PhD_QuantumMASALA/QuantumMASALA/src/qtm/dft/scf.py:418: ComplexWarning: Casting complex values to real discards the imaginary part
  vxc_arr[ik,iband] = hpsi_r#/ gwfn.reallat_dv


SCF Routine has exited
                          TIMERS                           
-----------------------------------------------------------
|            LABEL             |  CALL  |  TIME  | STATUS  |
-----------------------------------------------------------
-----------------------------------------------------------

                COUNTERS                 
-----------------------------------------
|            LABEL             | COUNT  |
-----------------------------------------
-----------------------------------------

[[-10.50315708 -11.36939725 -11.36939701 ...  -9.63525668  -9.63525635
   -7.7355693 ]
 [-10.57973657 -10.69759118 -11.13077338 ...  -9.19250514  -9.19251182
   -9.59911023]
 [-10.89780415 -10.22681951 -11.10567573 ...  -8.65528191  -8.6552829
   -9.1415997 ]
 ...
 [-10.83081092 -10.79460051 -10.25485879 ...  -9.01784025  -8.67086204
   -8.95993853]
 [-10.73213112 -10.93560893 -10.08289306 ...  -8.85761017  -9.90399847
   -9.07785932]
 [-10.57973645 -10.697591

In [4]:
l_wfn_kgrp[0][0].evl/ELECTRONVOLT_HART

array([-5.63639296,  6.47357234,  6.47357238,  6.47357242,  9.02632434,
        9.02632451,  9.02632453, 10.00436678, 14.22708703, 14.2270872 ,
       14.47107114, 17.75478405, 17.75478416, 17.75478422, 21.96831451,
       29.85344213, 29.85344227, 29.85344233, 30.98269702, 30.98269719,
       32.67081682, 32.67081697, 32.67081709, 35.95130562, 35.95130589,
       35.95130598, 41.86282945, 41.86282973, 41.86283009, 45.2875407 ])

In [5]:
kpts_q = KList(recilat=crystal.recilat, k_coords=kpts.k_cryst+np.array([[0,0,0.001]]).T, k_weights=np.ones(kpts.k_cryst.shape[1])/kpts.k_cryst.shape[0])

out_q = scf(dftcomm, crystal, kpts_q, grho, gwfn,
          numbnd, is_spin, is_noncolin,
          rho_start=mag_start, occ_typ=occ, smear_typ='gauss', e_temp=e_temp,
          conv_thr=conv_thr, diago_thr_init=diago_thr_init,
          iter_printer=print_scf_status)

scf_converged_q, rho_q, l_wfn_kgrp_q, en_q = out_q


print("Shifted SCF Routine has exited")
print(qtmlogger)

Iteration # 1, Run Time:   5.9 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1181e-02 Ry
Avg Diago Iterations: 7.7
Diago Threshold     : 1.00e-02 Ry

Total Energy:           -5.33847822 Ry

      one-el:           15.46709334 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89781265 eV
    LU Level:            6.86455467 eV
----------------------------------------

Iteration # 1, Run Time:   8.1 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1007e-02 Ry
Avg Diago Iterations: 2.5
Diago Threshold     : 7.63e-04 Ry

Total Energy:           -5.34521949 Ry

      one-el:           15.46035207 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89673910 eV
    LU Level:            6.86372559 eV
----------------------------------------

Iteration # 2, Run Time:  10

In [6]:
print(rho.data_g0)

[416.85324649-1.40840157e-50j]


In [7]:
l_wfn_kgrp_q[0][0].evl/ELECTRONVOLT_HART

array([-5.6363866 ,  6.47340204,  6.47354002,  6.47354003,  9.02635416,
        9.02635421,  9.02636947, 10.00444467, 14.22703302, 14.22703324,
       14.47111965, 17.75472324, 17.75487357, 17.75487376, 21.96839671,
       29.85326206, 29.85330953, 29.85330983, 30.9825386 , 30.98253886,
       32.67091592, 32.67091621, 32.67095914, 35.9513144 , 35.95149641,
       35.9514967 , 41.86260655, 41.86278312, 41.86278346, 45.28759844])

### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [8]:
# Imports
import numpy as np
import sys

sys.path.append(".")

dirname = "./si_4_gw/"

### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [9]:
from qtm.gw.io_bgw.epsinp import Epsinp

# Constructing input manually
# epsinp = Epsinp(epsilon_cutoff=1.2,
#                 use_wfn_hdf5=True,
#                 number_bands=8,
#                 write_vcoul=True,
#                 qpts=[[0.0,0.0,0.0]],
#                 is_q0=[True])

# Reading from epsilon.inp file
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')
# print(epsinp)

# There is an analogous system to read SigmaInp
from qtm.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
# print(sigmainp)

### Load WfnData
Calculation of dielectric matrix requires mean field eigenfunctions. \
Wavefunction data generated from mean-field codes can be read using the ``wfn2py`` utility, which assumes that the incoming data satisfies BerkeleyGW's [`wfn_h5`](http://manual.berkeleygw.org/3.0/wfn_h5_spec/) specification. The data is stored as a `NamedTuple` object.

For reasons discussed later, we also require wavefunctions on a shifted grid to calculate dielectric matrix at $q\to 0$. This shifted grid dataset will be referred to as `wfnqdata`.

Similarly, the utilities `read_rho` and `read_vxc` can be used to read density and exchange-correlation respectively.

In [10]:
# # wfn2py
from qtm.gw.io_bgw import inp
# from qtm.gw.io_bgw.wfn2py import wfn2py

# wfndata = wfn2py(dirname+'WFN.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
# # print(wfndata.__doc__)

# wfnqdata = wfn2py(dirname+'WFNq.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
# # print(wfnqdata.__doc__)

# # RHO data
rho_read = inp.read_rho(dirname+"RHO")

# # Vxc data
# vxc = inp.read_vxc(dirname+"vxc.dat")

In [11]:
print(rho)

<qtm.containers.field.get_FieldG.<locals>.FieldG object at 0x7efdb42640d0>


In [12]:
print(np.sum(np.linalg.norm(l_wfn_kgrp[0][0].evc_gk.data)**2))
print(l_wfn_kgrp[0][0].evc_gk.data[0][:20])

29.999999999999982
[ 7.52163396e-01+5.82657980e-01j  7.95827174e-02+6.16481686e-02j
  2.43670726e-03+1.88755401e-03j  4.15943037e-03+3.22207440e-03j
  7.59986939e-04+5.88722939e-04j  7.59976933e-04+5.88706517e-04j
  4.15944809e-03+3.22208097e-03j  2.43666248e-03+1.88754439e-03j
  7.95826984e-02+6.16481706e-02j  7.95827048e-02+6.16481780e-02j
  4.46317036e-09-3.32229492e-09j  1.15147511e-02+8.91980670e-03j
  7.12214775e-03+5.51712091e-03j  8.10274039e-04+6.27655155e-04j
  1.88743480e-04+1.46201174e-04j  1.92356352e-04+1.48998972e-04j
 -7.05416504e-03-5.46447676e-03j -8.49503572e-03-6.58064875e-03j
  2.43663686e-03+1.88750118e-03j  1.15146680e-02+8.91988262e-03j]


### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [13]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon
from qtm.klist import KList

kpts_gw =   KList(recilat=kpts.recilat,   cryst=kpts.k_cryst.T,   weights=kpts.k_weights)
kpts_gw_q = KList(recilat=kpts_q.recilat, cryst=kpts_q.k_cryst.T, weights=kpts_q.k_weights)

# Manual initialization
epsilon = Epsilon(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq = kpts_gw_q,        
    l_wfn = l_wfn_kgrp,
    l_wfnq = l_wfn_kgrp_q,
    # l_gsp_wfn = wfndata.l_gk,
    # l_gsp_wfnq = wfnqdata.l_gk,
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 16354.05it/s]


The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix

<!-- 1.  ``matrix_elements``: Calculation of Planewave Matrix elements
    $$M_{nn'}({\textbf k},{\textbf q},{\textbf G}) = \bra{n\,{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n'\,\textbf k}$$
    where the $\textbf G$-vectors included in the calculation satisfy $|\textbf q + \textbf G|^2 < E_{\text{cut}}$.
    Since this is a convolution in k-space, the time complexity can be reduced from $\mathcal{O}\left(N^2_G\right)$ to $\mathcal{O}\left(N_G\ln N_G\right)$ by using Fast Fourier Transform, where $N_G$  the number of reciprocal lattice vectors in the wavefunction.
    $$
    M_{nn'}({\bf k},{\bf q},\{{\bf G}\}) = {\rm FFT}^{-1}\left( \phi^{*}_{n,{\bf k}+{\bf q} }({\bf r}) \phi_{n',{\bf k} }({\bf r}) \right).
    $$
    where $\phi_{n',{\bf k}}({\bf r}) = {\rm FFT}\left( \psi_{n\bf k}(\bf G)\right)$. 
    
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
    $$
        P_{\textbf{GG'}}{\left({\textbf q}\;\!;0\right)}=
        \,\,{}\sum_{n}^{\textrm occ}\sum_{n'}^{\textrm emp}\sum_{{\textbf k}}
        \frac{
        \bra{n'\textbf k}e^{-i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n{\textbf k}{+}{\textbf q}}
        \bra{n{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G'})\cdot{\textbf r}}\ket{n'\textbf k}
        }{E_{n{\textbf k}{+}{\textbf q}}\,{-}\,E_{n'{\textbf k}}}.
    $$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix
    $$
        \epsilon_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}=
        \delta_{\textbf{GG'}}\,{-}\,v{\left({\textbf q}{+}{\textbf G}\right)} \,
        P_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}
    $$
    where $ v(\textbf{q} + \textbf{G}) = \frac{8\pi}{\left|\textbf q + \textbf G\right|^2} $ is bare Coulomb potential, written in Rydberg units. If this formula is used as-is for the case where $|\textbf q| = |\textbf G| = 0$, the resulting $v\left({\textbf{q=0}, \textbf{G=0}}\;\!\right)$ blows up as $1/q^2$. However, for 3D gapped systems, the matrix elements $\big| M_{nn'}\left({\bf k},{\textbf{q}\to\textbf{0}},{\textbf{G=0}}\right)\big| \sim q$ cancel the Coulomb divergence and $\epsilon_{\textbf{00}}\left({\textbf q\to\textbf{0}}\;\!\right) \sim q^2/q^2$ which is a finite number. In order to calculate $\epsilon_{\textbf{00}}\left({\textbf q=\textbf{0}}\;\!\right)$, we use the scheme specified in BGW2012, wherein $q=0$ is replaced with a small but non-zero value. Since matrix element calculation involves the eigenvectors $\ket{n{\textbf k}{+}{\textbf q}}$, having a non-$\Gamma$-centered $\textbf q\to 0$ point requires mean-field eigenvectors over a shifted $k$-grid. -->

In [14]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))
        # if i_q==0:
        #     N=10
        #     print(M.shape)
        #     print(M[0,0,0,:N].real)
        #     break

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # chimat/=2
        
        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            print("q0")
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        # std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))
        std_eps = np.std(np.abs(epsref) - np.abs(epsinv)) / np.sqrt(np.prod(list(epsinv.shape)))
        

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")
            N_ELE_SHOW=10
            # print(chimat.diagonal()[-N_ELE_SHOW:].real)
            print(np.max(np.abs(epsinv-epsref)))
            indices = np.where(np.abs(epsinv)-np.abs(epsref)>1e-5)
            
            print(len(indices[0]))
            # print((epsinv+epsref)[indices])
            # print(epsref[indices])
            # print(epsinv[indices])
            # print(epsref.diagonal()[-N_ELE_SHOW:].real)
            # print(epsinv.diagonal()[-N_ELE_SHOW:].real)
            # print((epsinv-epsref).diagonal()[-N_ELE_SHOW:].real)

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

# epsinp.no_min_fftgrid = False
# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
# calculate_epsilon()


Epsilon> q-pt index:   0%|          | 0/64 [00:00<?, ?it/s]

Epsilon> q-pt index:   2%|▏         | 1/64 [00:00<00:19,  3.17it/s]

q0
Standard deviation exceeded 1e-16 tolerance: 3.7093720825876787e-07, for i_q:0
42.21657800193814
67


Epsilon> q-pt index:   3%|▎         | 2/64 [00:00<00:20,  2.99it/s]

Standard deviation exceeded 1e-16 tolerance: 3.1277758703721435e-10, for i_q:1
0.4207231962765918
0


...

Standard deviation exceeded 1e-16 tolerance: 3.0704700235083336e-10, for i_q:61
0.2899578341400892
0


Epsilon> q-pt index:  98%|█████████▊| 63/64 [00:23<00:00,  2.73it/s]

Standard deviation exceeded 1e-16 tolerance: 3.2951484027281764e-10, for i_q:62
0.2804061854431803
0


Epsilon> q-pt index: 100%|██████████| 64/64 [00:23<00:00,  2.73it/s]

Standard deviation exceeded 1e-16 tolerance: 3.1055962035802695e-10, for i_q:63
0.4207232423273962
0


### Sigma Calculation

In [15]:
print(rho.data_g0)
print(crystal.numel)
rho._data *= crystal.numel / (sum(rho.data_g0))
print(rho._data)

[416.85324649-1.40840157e-50j]
8
[[ 8.        -1.23299441e-68j  1.77749909+9.69909921e-16j
   0.36034418+3.77788549e-16j ... -0.08258707+1.30010401e-16j
  -0.07425604+6.48443770e-17j  1.77749909-1.46955634e-16j]]


In [16]:
from collections import namedtuple
from qtm.gw.sigma import Sigma

outdir = dirname+"temp/"

rho._data *= crystal.numel / (sum(rho.data_g0))# * rho.gspc.reallat_dv)

rho_nt = namedtuple("RHO", ["rho", "gvecs"])(rho.data[0], rho.gspc.g_cryst.T)
vxc_nt = namedtuple("VXC", ["vxc"])(vxc/ELECTRONVOLT_HART)


sigma = Sigma(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq=kpts_gw_q,
    l_wfn = [wfn[0] for wfn in l_wfn_kgrp],
    l_wfnq = [wfn[0] for wfn in l_wfn_kgrp_q],
    l_gsp_wfn = [wfn[0].gkspc for wfn in l_wfn_kgrp],
    l_gsp_wfnq = [wfn[0].gkspc for wfn in l_wfn_kgrp_q],
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    sigmainp=sigmainp,
    epsinp = epsinp,
    # l_epsmats=l_epsmats,
    l_epsmats=epsilon.l_epsinv,
    rho=rho_nt,
    vxc=vxc_nt
)

# sigma = Sigma.from_data(
#     wfndata=wfndata,
#     wfnqdata=wfnqdata,
#     sigmainp=sigmainp,
#     epsinp=epsinp,
#     l_epsmats=epsilon.l_epsinv,
#     rho=rho,
#     vxc=vxc,
#     outdir=outdir,
# )

Vcoul calculation for qpts:   0%|          | 0/64 [00:00<?, ?it/s]

vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7efdb455f450>
        * qpts = <qtm.gw.core.QPoints object at 0x7efd7b478990>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [00:49<00:00,  1.30it/s]


In [17]:
sigma_sx_cohsex_mat = sigma.sigma_sx_static(yielding=True)    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(sigma_sx_cohsex_mat)

Sigma_SX_Static:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_Static:   6%|▋         | 4/64 [00:00<00:01, 31.95it/s]

(137,)


Sigma_SX_Static: 100%|██████████| 64/64 [00:01<00:00, 32.28it/s]

Sigma SX COHSEX
   11.875835   10.672206   11.328765
    8.030570   10.678362   10.014678
    8.031743    8.719808    8.366404
    8.031970    8.719533    8.365541
    2.910472    2.586913    2.872630
    2.908779    2.587801    2.426855
    2.911732    1.328987    2.426960
    2.681682    1.328812    0.739609


from tqdm.auto import tqdm
sigma_x_mat = sigma.sigma_x(yielding=True)    
print("Sigma X")
sigma.pprint_sigma_mat(sigma_x_mat)

In [18]:
sigma_ch_cohsex_mat = sigma.sigma_ch_static()    
print("Sigma CH COHSEX")
sigma.pprint_sigma_mat(sigma_ch_cohsex_mat)

Sigma_CH_Static_Partial:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:03<00:00, 17.97it/s]

Sigma CH COHSEX
   -6.627248   -6.303135   -6.494451
   -5.099279   -6.303097   -6.068627
   -5.099294   -5.571196   -5.407666
   -5.099536   -5.571210   -5.406884
   -4.746137   -4.701091   -4.917357
   -4.745580   -4.701259   -4.287270
   -4.746849   -3.460434   -4.287115
   -4.757445   -3.460220   -2.975094


In [19]:
sigma.autosave=False
sigma.print_condition=True
cohsex_result = sigma.calculate_static_cohsex()

Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_Static:   6%|▋         | 4/64 [00:00<00:01, 31.85it/s]

Sigma X
  -17.549889  -16.038831  -16.918927
  -12.974926  -16.038804  -14.908715
  -12.974823  -13.417574  -13.204511
  -12.975040  -13.417537  -13.204406
   -5.765599   -5.196199   -5.934483
   -5.765459   -5.196466   -5.081723
   -5.765406   -3.789636   -5.081648
   -5.831193   -3.789591   -2.406333
(137,)


Sigma_CH_Static_Partial:   3%|▎         | 2/64 [00:00<00:03, 18.09it/s]

Sigma SX STATIC
   11.875835   10.672206   11.328765
    8.030570   10.678362   10.014678
    8.031743    8.719808    8.366404
    8.031970    8.719533    8.365541
    2.910472    2.586913    2.872630
    2.908779    2.587801    2.426855
    2.911732    1.328987    2.426960
    2.681682    1.328812    0.739609


Sigma_CH_Static_Exact:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma CH STATIC
   -6.627248   -6.303135   -6.494451
   -5.099279   -6.303097   -6.068627
   -5.099294   -5.571196   -5.407666
   -5.099536   -5.571210   -5.406884
   -4.746137   -4.701091   -4.917357
   -4.745580   -4.701259   -4.287270
   -4.746849   -3.460434   -4.287115
   -4.757445   -3.460220   -2.975094


Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:22<00:00,  2.82it/s]

Sigma CH EXACT STATIC
   -7.444555   -7.623042   -7.626662
   -7.756611   -7.623304   -7.314288
   -7.757066   -7.477268   -7.674453
   -7.757444   -7.476981   -7.673776
   -7.335210   -6.794909   -7.354451
   -7.334659   -6.795262   -7.120472
   -7.335679   -7.674156   -7.120465
   -7.829751   -7.673838   -6.232123
Sig (Exact):
  -13.118609  -12.989666  -13.216825
  -12.700966  -12.983746  -12.208325
  -12.700146  -12.175034  -12.512559
  -12.700515  -12.174985  -12.512641
  -10.190337   -9.404194  -10.416305
  -10.191339   -9.403927   -9.775341
  -10.189354  -10.134804   -9.775154
  -10.979262  -10.134617   -7.898847
Eqp0 (Exact):
   -8.251845   -3.519778   -5.575222
    5.142004   -3.513858   -2.586362
    5.142824    2.014844    3.845879
    5.142454    2.014893    3.845797
    8.897207    6.723030    7.761276
    8.896205    6.723297    9.693525
    8.898191   17.164045    9.693713
    9.966476   17.164233   14.034580
Sig (Partial):
  -12.301302  -11.669759  -12.084614
  -10.04363

In [20]:
from qtm.gw.io_bgw.sigma_hp_reader import read_sigma_hp
ref_dict = read_sigma_hp(dirname+"sigma_hp_cohsex.log")
for ik in cohsex_result:
    print("k-point index:",ik)
    qtty = 'Eqp1'
    print(np.abs(ref_dict[ik+1][qtty]-np.around(cohsex_result[ik][qtty],6)))

k-point index: 0
[0.001689 0.001437 0.003178 0.001968 0.000374 0.001056 0.000793 0.001819]
k-point index: 40
[2.330e-04 4.371e-03 1.704e-03 1.575e-03 9.070e-04 5.400e-05 6.870e-04
 1.565e-03]
k-point index: 8
[0.002168 0.00098  0.002048 0.002027 0.001034 0.000637 0.000463 0.000987]


In [21]:
sigma.print_condition=True
sigma_ch_exact_mat = sigma.sigma_ch_static_exact()    
print("Sigma CH COHSEX EXACT")
sigma.pprint_sigma_mat(sigma_ch_exact_mat)

Sigma_CH_Static_Exact:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:22<00:00,  2.82it/s]

Sigma CH COHSEX EXACT
   -7.444555   -7.623042   -7.626662
   -7.756611   -7.623304   -7.314288
   -7.757066   -7.477268   -7.674453
   -7.757444   -7.476981   -7.673776
   -7.335210   -6.794909   -7.354451
   -7.334659   -6.795262   -7.120472
   -7.335679   -7.674156   -7.120465
   -7.829751   -7.673838   -6.232123


In [22]:
rho.data

array([[ 8.        +6.90124257e-85j,  1.77749909+9.69909921e-16j,
         0.36034418+3.77788549e-16j, ..., -0.08258707+1.30010401e-16j,
        -0.07425604+6.48443770e-17j,  1.77749909-1.46955634e-16j]])

In [23]:
sigma.print_condition=False
sigma_sx_gpp = sigma.sigma_sx_gpp()    
print("Sigma SX GPP")
sigma.pprint_sigma_mat(sigma_sx_gpp)

Sigma_SX_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_GPP: 100%|██████████| 64/64 [00:48<00:00,  1.32it/s]

Sigma SX GPP
   12.244002   10.838113   11.549690
    8.177149   10.846066   10.174693
    8.177726    8.838469    8.501843
    8.179000    8.837931    8.500342
    3.296765    2.864992    3.299154
    3.293126    2.866973    2.790099
    3.298240    1.738082    2.791498
    3.371348    1.737541    1.089234


In [24]:
sigma.print_condition=False
sigma_ch_gpp,_ = sigma.sigma_ch_gpp()    
print("Sigma CH GPP")
sigma.pprint_sigma_mat(sigma_ch_gpp)

Sigma_CH_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_GPP: 100%|██████████| 64/64 [00:53<00:00,  1.21it/s]

Sigma CH GPP
   -5.796970   -5.847274   -5.879221
   -5.403142   -5.848742   -5.642406
   -5.402436   -5.559447   -5.573669
   -5.403744   -5.559200   -5.572457
   -5.340712   -4.982012   -5.446887
   -5.337889   -4.983754   -4.889794
   -5.341954   -4.622041   -4.891018
   -5.723835   -4.621603   -3.771267


In [25]:
gpp_result = sigma.calculate_gpp()

Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_X: 100%|██████████| 64/64 [00:01<00:00, 33.07it/s]

Sigma X GPP



Sigma_CH_Static_Partial:   3%|▎         | 2/64 [00:00<00:03, 17.29it/s]

  -17.549889  -16.038831  -16.918927
  -12.974926  -16.038804  -14.908715
  -12.974823  -13.417574  -13.204511
  -12.975040  -13.417537  -13.204406
   -5.765599   -5.196199   -5.934483
   -5.765459   -5.196466   -5.081723
   -5.765406   -3.789636   -5.081648
   -5.831193   -3.789591   -2.406333


...

Sigma CH GPP
   -5.796970   -5.847274   -5.879221
   -5.403142   -5.848742   -5.642406
   -5.402436   -5.559447   -5.573669
   -5.403744   -5.559200   -5.572457
   -5.340712   -4.982012   -5.446887
   -5.337889   -4.983754   -4.889794
   -5.341954   -4.622041   -4.891018
   -5.723835   -4.621603   -3.771267
Sig GPP:
  -11.511511  -11.707945  -11.814563
  -11.529584  -11.701583  -10.999258
  -11.528419  -11.091587  -11.409730
  -11.528738  -11.091692  -11.409967
   -9.104082   -8.360128   -9.300763
   -9.104762   -8.360249   -8.598020
   -9.103535   -8.780456   -8.597844
   -9.719833   -8.780463   -6.716880
Eqp0



Sigma_CH_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

   -6.644747   -2.238057   -4.172961
    6.313386   -2.231695   -1.377295
    6.314550    3.098291    4.948708
    6.314231    3.098186    4.948471
    9.983462    7.767096    8.876818
    9.982782    7.766975   10.870846
    9.984009   18.518394   10.871022
   11.225904   18.518387   15.216547
dE [[0.07349864 0.07349864 0.07349864 0.07349864 0.07349864 0.07349864
  0.07349864 0.07349864]
 [0.07349864 0.07349864 0.07349864 0.07349864 0.07349864 0.07349864
  0.07349864 0.07349864]
 [0.07349864 0.07349864 0.07349864 0.07349864 0.07349864 0.07349864
  0.07349864 0.07349864]]


Sigma_SX_GPP: 100%|██████████| 64/64 [00:49<00:00,  1.30it/s]

Sigma CH GPP dE
   -6.056252   -6.124192   -6.149626
   -5.636946   -6.125780   -5.916333
   -5.643259   -5.843641   -5.861614
   -5.637879   -5.843407   -5.860377
   -5.567695   -5.285952   -5.772181
   -5.564350   -5.287811   -5.099396
   -5.568952   -4.909145   -5.099706
   -5.940856   -4.908528   -4.062506
Sigma SX GPP dE
   -6.056252   -6.124192   -6.149626
   -5.636946   -6.125780   -5.916333
   -5.643259   -5.843641   -5.861614
   -5.637879   -5.843407   -5.860377
   -5.567695   -5.285952   -5.772181
   -5.564350   -5.287811   -5.099396
   -5.568952   -4.909145   -5.099706
   -5.940856   -4.908528   -4.062506
Z:
    0.714876    0.775221    0.755487
    0.839964    0.775062    0.778938
    0.839974    0.817146    0.829061
    0.839959    0.817134    0.829068
    0.830710    0.835606    0.833228
    0.830711    0.835579    0.839639
    0.830701    0.812220    0.839640
    0.834789    0.812135    0.837510
Sig_2:
[[-11.501702-0.00e+00j -11.337947+1.04e-04j -11.572108-0.00e+00j]
 [-1

In [26]:
from qtm.gw.io_bgw.sigma_hp_reader import read_sigma_hp

ref_dict = read_sigma_hp(dirname+"../gpp/sigma_hp.log")
for ik in gpp_result:
    print("k-point index:",ik)
    qtty = 'Eqp1'
    print(np.abs(ref_dict[ik+1][qtty]-np.around(gpp_result[ik][qtty],6)))

FileNotFoundError: [Errno 2] No such file or directory: './si_4_gw/../gpp/sigma_hp.log'

### Load WfnData
Calculation of dielectric matrix requires mean field eigenfunctions. \
Wavefunction data generated from mean-field codes can be read using the ``wfn2py`` utility, which assumes that the incoming data satisfies BerkeleyGW's [`wfn_h5`](http://manual.berkeleygw.org/3.0/wfn_h5_spec/) specification. The data is stored as a `NamedTuple` object.

For reasons discussed later, we also require wavefunctions on a shifted grid to calculate dielectric matrix at $q\to 0$. This shifted grid dataset will be referred to as `wfnqdata`.

Similarly, the utilities `read_rho` and `read_vxc` can be used to read density and exchange-correlation respectively.

### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [ ]:
from quantum_masala.gw.core import QPoints
from quantum_masala.gw.epsilon import Epsilon

epsilon = Epsilon(
    crystal = crystal,
    gspace = gspc_rho,
    kpts = kpts,
    kptsq = kpts_shifted,
    l_wfn = l_wfn_kgrp,
    l_wfnq = l_wfn_kgrp_q,
    l_gsp_wfn = [wfn.gkspc for wfn in l_wfn_kgrp],
    l_gsp_wfnq = [wfn.gkspc for wfn in l_wfn_kgrp_q],
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
3.  ``epsilon_inverse``: Calculation of (static) Epsilon Inverse matrix

In [ ]:
from tqdm import trange
from quantum_masala.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


np.set_printoptions(formatter={'float': lambda x: "{0:0.30f}".format(x)})

def calculate_epsilon(numq=None, writing=True):
    epsmats = []
    res = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
            indices_gspace_sorted = sort_cryst_like_BGW(
                cryst=gkspc.g_cryst, key_array=key
            )
        else:
            key = gkspc.norm2
            indices_gspace_sorted = sort_cryst_like_BGW(
                cryst=gkspc.g_cryst, key_array=key
            )

        # Calculate polarizability matrix (Memory-inefficient, but faster)
        chimat = epsilon.polarizability(next(epsilon.matrix_elements(i_q=i_q)))

        # chimat /= 1037729756552.8315
        # chimat *= 9.63652239202381036380e-13
        chimat/=2.0

        # print(np.real(chimat[0,::20]))

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.calculate_polarizability(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)

        # indices = epsilon.l_gq[i_q].gk_indices_tosorted
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv

        # print((epsinv[0,::20]))

        res.append(epsinv)

        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            # indices = epsilon.l_gq[i_q].gk_indices_tosorted
            if writing:
                epsilon.write_epsmat(
                    filename="si_4_gw/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        # print("epsinv.shape", epsinv.shape,"epsref.shape", epsref.shape)
        mindim = min(*epsref.shape)#, *epsinv.shape)
        epsref = epsref[:mindim, :mindim]
        std_eps = np.std(np.abs(epsref) - np.abs(epsinv)) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-15
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance", std_eps)
            print("i_q", i_q)

    if writing:
        epsilon.write_epsmat(filename="si_4_gw/epsmat_qtm.h5", epsinvmats=epsmats)

    return res

epsinp.no_min_fftgrid = True
epsilon = Epsilon(
    crystal = crystal,
    gspace = gspc_rho,
    kpts = kpts,
    kptsq = kpts_shifted,
    l_wfn = l_wfn_kgrp,
    l_wfnq = l_wfn_kgrp_q,
    l_gsp_wfn = [wfn.gkspc for wfn in l_wfn_kgrp],
    l_gsp_wfnq = [wfn.gkspc for wfn in l_wfn_kgrp_q],
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

# print(epsilon.l_wfn[0].evc_gk[0,0,:10])

# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
l_epsmats = calculate_epsilon()
#numq=1)

# epsinp.no_min_fftgrid = False
# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
# calculate_epsilon()#numq=1)


### Sigma Calculation

In [ ]:
print(rho.data.shape)
print(rho.gspc.grid_shape)
print(rho.gspc.cryst.shape)

In [ ]:
from quantum_masala.gw.io_bgw import inp


vxc = inp.read_vxc(dirname+"vxc.dat")
print(vxc.__doc__)
print(np.array(vxc.vxc).shape)
print(v_xc.data.shape)

In [ ]:
from collections import namedtuple
from quantum_masala.gw.io_bgw import inp
from quantum_masala.gw.io_bgw.sigmainp import Sigmainp
from quantum_masala.gw.sigma import Sigma


# dirname = "./scripts/results/si_6_nband272_pristine_cohsex/si_6_gw/"
# outdir = "./test/tempdir_20230618_173806/"

rho_nt = namedtuple("RHO", ["rho", "gvecs"])(rho.data[0], rho.gspc.cryst.T)

# RHO data
# rho = inp.read_rho(dirname+"RHO")

# Vxc data
# vxc = inp.read_vxc(dirname+"vxc.dat")


# There is an analogous system to read SigmaInp
# from quantum_masala.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
print(sigmainp)

sigma = Sigma(
    crystal = crystal,
    gspace = gspc_rho,
    kpts = kpts,
    kptsq=kpts_shifted,
    l_wfn = l_wfn_kgrp,
    l_wfnq = l_wfn_kgrp_q,
    l_gsp_wfn = [wfn.gkspc for wfn in l_wfn_kgrp],
    l_gsp_wfnq = [wfn.gkspc for wfn in l_wfn_kgrp_q],
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    sigmainp=sigmainp,
    epsinp = epsinp,
    # l_epsmats=l_epsmats,
    l_epsmats=epsilon.l_epsinv,
    rho=rho_nt,
    vxc=v_xc
)

# sigma = Sigma.from_data(
#     wfndata=wfndata,
#     wfnqdata=wfnqdata,
#     sigmainp=sigmainp,
#     epsinp=epsinp,
#     l_epsmats=epsilon.l_epsinv,
#     rho=rho_nt,
#     vxc=vxc,
#     outdir=dirname+"temp/",
# )

In [ ]:
sigma.qpts.cryst

In [ ]:
np.set_printoptions(precision=4)
sigma.print_condition=False
sigma_x_mat = sigma.sigma_x()    
print("Sigma X GPP")
sigma.pprint_sigma_mat(np.real(sigma_x_mat.T))

In [ ]:
sigma.print_condition=False
sigma_sx_cohsex_mat = sigma.sigma_sx_static()    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(np.real(sigma_sx_cohsex_mat.T))

In [ ]:
sigma.print_condition=True
sigma_sx_gpp_mat = sigma.sigma_sx_gpp()    
print("Sigma SX GPP")
sigma.pprint_sigma_mat(np.real(sigma_sx_gpp_mat.T))

In [ ]:
sigma.calculate_gpp()

In [ ]:
sigma.calculate_static_cohsex()

In [ ]:
print(sigma.rho.gvecs.shape)

In [ ]:
v_xc.shape

In [ ]:
sigma.print_condition=False
dE = 1.0 / sigma.ryd
sigma_ch_gpp_mat = sigma.sigma_ch_gpp(dE)    
print("Sigma CH GPP")
sigma.pprint_sigma_mat(np.real(sigma_ch_gpp_mat[1].T))